In [1]:
import calendar
import pandas as pd
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 3, 19)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

In [3]:
format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [4]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,22,S,AIMIRT,10000,13.1,0,DOS,SET,2022-03-14


In [5]:
names = orders['name']
name = names.to_string(index=False)

In [6]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,AIMIRT,0.2210,0.2207,0.2205,0.2200,0.8822,"10,000",2022-03-14,2022-03-28,,0


In [7]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'AIMIRT'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,719,AIMIRT,13.40,11.61,I,0.00,13.10,3.47,0.03,13.00,"10,000",-4,4,"10,000",0,0,DOS,SET


### Create orders from stocks after adjust stocks in port_lite from consensus

In [8]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

23

In [9]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy

,trade,name,qty,price,active,reason,market
0,O,AMATA,0,0.00,0,,SET100
1,O,ASIAN,9000,15.20,0,,SET
2,O,ASK,3000,42.00,0,1L,SET
3,O,BAM,10000,20.50,0,,SET100
4,O,BAY,0,0.00,0,,SET
5,O,BBL,2400,130.00,0,1L,SET50
6,O,BDMS,12000,23.20,0,1L,SET50
7,O,CHG,60000,3.36,0,,SET100
8,O,DCC,45000,2.80,0,6L,SET
9,O,EA,0,0.00,0,,SET50


In [10]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell

,trade,name,qty,price,active,reason,market
0,I,JASIF,10000,11.30,0,DOS,SET
1,I,HREIT,40100,9.00,0,DOS,SET
2,I,RATCH,6000,46.00,0,DOS,SET50
3,I,SYNEX,5000,30.00,0,5pct,SET100
4,I,BCH,5000,21.40,0,CP1S,SET100
5,I,DOHOME,4800,23.30,0,15pct,SET100
6,I,PTT,3000,41.25,0,5pct,SET50
7,I,SINGER,1500,50.75,0,20pct,SET100
8,I,KCE,1000,68.00,0,CUT,SET50
9,I,IP,2500,20.30,0,CP1S,mai


In [11]:
df_order = df_buy.append(df_sell, ignore_index=True)
df_order.shape

(45, 7)

In [12]:
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,5,5,5,5,5,5
I,15,15,15,15,15,15
O,25,25,25,25,25,25


In [13]:
df_order.loc[df_order.trade == 'I','trade'] = 'S'
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,5,5,5,5,5,5
O,25,25,25,25,25,25
S,15,15,15,15,15,15


In [14]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(81, 2)

In [15]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(81, 9)

### If there is no dividend record, must create dividend record before proceed

In [16]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge


In [17]:
mask = df_merge['_merge'] == 'both'
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,5,5,5,5,5,5,5
O,25,25,25,25,25,25,25
S,15,15,15,15,15,15,15


### Set xdate 2 weeks window of first Tuesday and second Friday

In [18]:
fm_date = '2022-03-22'
to_date = '2022-04-15'

### Restart and Run All Cells

In [19]:
fltr_sts_o = (df_orders.trade == 'O') & (df_orders.xdate >= fm_date) & (df_orders.xdate <= to_date) 
df_sts_o = df_orders[fltr_sts_o]
df_sts_o

,trade,name,qty,price,active,reason,market,xdate
8,O,DCC,45000,2.8,0,6L,SET,2022-04-05


In [20]:
fltr_sts_b = (df_orders.trade == 'B') 
df_sts_b = df_orders[fltr_sts_b]
df_sts_b

,trade,name,qty,price,active,reason,market,xdate
15,B,MAKRO,1500,39.75,0,RD05pct,SET,2022-03-03
19,B,RCL,3000,48.25,0,DOS,SET100,2022-03-23
23,B,STA,15000,26.25,0,RD20pct,SET100,2022-04-19
26,B,TISCO,500,97.75,0,ROUND,SET50,2022-04-29
27,B,TMT,15000,10.50,0,ROUND,SET,2022-04-18


In [21]:
fltr_sts_s = (df_orders.trade == 'S')
df_sts_s = df_orders[fltr_sts_s]
df_sts_s.sort_values('name',ascending=True)

,trade,name,qty,price,active,reason,market,xdate
44,S,AIMIRT,10000,13.10,0,DOS,SET,2022-03-14
42,S,ASP,30000,3.82,0,DOS,SET,2022-03-10
34,S,BCH,5000,21.40,0,CP1S,SET100,2022-05-10
41,S,CPNCG,20000,12.50,0,DOS,SET,2022-02-14
35,S,DOHOME,4800,23.30,0,15pct,SET100,2022-02-02
31,S,HREIT,40100,9.00,0,DOS,SET,2022-03-04
39,S,IP,2500,20.30,0,CP1S,mai,2022-02-02
30,S,JASIF,10000,11.30,0,DOS,SET,2022-03-03
38,S,KCE,1000,68.00,0,CUT,SET50,2022-03-21
36,S,PTT,3000,41.25,0,5pct,SET50,2022-03-03


In [22]:
frames = [df_sts_b, df_sts_o, df_sts_s]
df_orders = pd.concat(frames, sort=False)
df_orders.sort_values(['trade','name'],ascending=True)

,trade,name,qty,price,active,reason,market,xdate
15,B,MAKRO,1500,39.75,0,RD05pct,SET,2022-03-03
19,B,RCL,3000,48.25,0,DOS,SET100,2022-03-23
23,B,STA,15000,26.25,0,RD20pct,SET100,2022-04-19
26,B,TISCO,500,97.75,0,ROUND,SET50,2022-04-29
27,B,TMT,15000,10.50,0,ROUND,SET,2022-04-18
8,O,DCC,45000,2.80,0,6L,SET,2022-04-05
44,S,AIMIRT,10000,13.10,0,DOS,SET,2022-03-14
42,S,ASP,30000,3.82,0,DOS,SET,2022-03-10
34,S,BCH,5000,21.40,0,CP1S,SET100,2022-05-10
41,S,CPNCG,20000,12.50,0,DOS,SET,2022-02-14


In [23]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,6,6,6,6,6,6,6
S,15,15,15,15,15,15,15


In [24]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

In [25]:
rcds = df_orders.values.tolist()
len(rcds)

21

In [26]:
for rcd in rcds:
    print(rcd)

['B', 'MAKRO', 1500, 39.75, 0, 'RD05pct', 'SET', '2022-03-03']
['B', 'RCL', 3000, 48.25, 0, 'DOS', 'SET100', '2022-03-23']
['B', 'STA', 15000, 26.25, 0, 'RD20pct', 'SET100', '2022-04-19']
['B', 'TISCO', 500, 97.75, 0, 'ROUND', 'SET50', '2022-04-29']
['B', 'TMT', 15000, 10.5, 0, 'ROUND', 'SET', '2022-04-18']
['B', 'DCC', 45000, 2.8, 0, '6L', 'SET', '2022-04-05']
['S', 'JASIF', 10000, 11.3, 0, 'DOS', 'SET', '2022-03-03']
['S', 'HREIT', 40100, 9.0, 0, 'DOS', 'SET', '2022-03-04']
['S', 'RATCH', 6000, 46.0, 0, 'DOS', 'SET50', '2022-03-17']
['S', 'SYNEX', 5000, 30.0, 0, '5pct', 'SET100', '2022-03-10']
['S', 'BCH', 5000, 21.4, 0, 'CP1S', 'SET100', '2022-05-10']
['S', 'DOHOME', 4800, 23.3, 0, '15pct', 'SET100', '2022-02-02']
['S', 'PTT', 3000, 41.25, 0, '5pct', 'SET50', '2022-03-03']
['S', 'SINGER', 1500, 50.75, 0, '20pct', 'SET100', '2022-04-29']
['S', 'KCE', 1000, 68.0, 0, 'CUT', 'SET50', '2022-03-21']
['S', 'IP', 2500, 20.3, 0, 'CP1S', 'mai', '2022-02-02']
['S', 'WHART', 20000, 12.4, 0, 'DO

In [27]:
sql = """
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [28]:
for rcd in rcds:
    conlite.execute(sql, rcd)

### After call Ord-log.bat

In [32]:
def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [33]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df[colt], layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trans    name  ..., width=800)

In [34]:
df[colt]

,trans,name,spd,reason,qty,target,current,percent,active,xdate
0,Buy,DCC,-8,6L,45000,2.80,2.96,-0.67%,0,2022-04-05
1,Buy,MAKRO,-6,RD05pct,1500,39.75,41.25,-0.60%,0,2022-03-03
2,Buy,RCL,-9,DOS,3000,48.25,50.50,-1.46%,0,2022-03-23
3,Buy,STA,-4,RD20pct,15000,26.25,27.25,-2.68%,0,2022-04-19
4,Buy,TISCO,-7,ROUND,500,97.75,99.50,-0.50%,0,2022-04-29
5,Buy,TMT,-7,ROUND,15000,10.50,11.20,0.00%,0,2022-04-18
6,Sell,AIMIRT,2,DOS,10000,13.10,12.90,0.00%,0,2022-03-14
7,Sell,ASP,17,DOS,30000,3.82,3.48,0.00%,0,2022-03-10
8,Sell,BCH,2,CP1S,5000,21.40,21.20,+0.95%,0,2022-05-10
9,Sell,CPNCG,2,DOS,20000,12.50,12.30,0.00%,0,2022-02-14
